In [1]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# -----------------------------
# 1. Load Your WAV File
# -----------------------------
speech, sr = librosa.load("Sample-WAV-File.wav", sr=None)

print("Sampling Rate:", sr)
print("Total Samples:", len(speech))
print("Duration (sec):", len(speech)/sr)

# -----------------------------
# 2. Add White Noise (10 dB SNR)
# -----------------------------
def add_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10**(snr_db/10)
    noise_power = signal_power / snr_linear
    
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    noisy_signal = signal + noise
    return noisy_signal

noisy_speech = add_noise(speech, 10)

# -----------------------------
# 3. SNR Calculation Function
# -----------------------------
def compute_snr(clean, noisy):
    noise = noisy - clean
    return 10 * np.log10(np.sum(clean**2) / np.sum(noise**2))

snr_before = compute_snr(speech, noisy_speech)
print("SNR Before Enhancement:", snr_before, "dB")

# -----------------------------
# 4. Spectral Subtraction
# -----------------------------
n_fft = 1024
hop_length = 512

# STFT of noisy speech
stft_noisy = librosa.stft(noisy_speech, n_fft=n_fft, hop_length=hop_length)
magnitude = np.abs(stft_noisy)
phase = np.angle(stft_noisy)

# Estimate noise spectrum (first 5 frames)
noise_estimate = np.mean(magnitude[:, :5], axis=1, keepdims=True)

# Spectral subtraction
enhanced_magnitude = magnitude - noise_estimate
enhanced_magnitude = np.maximum(enhanced_magnitude, 0.001)

# Reconstruct enhanced signal
enhanced_stft = enhanced_magnitude * np.exp(1j * phase)
enhanced_speech = librosa.istft(enhanced_stft, hop_length=hop_length)

# -----------------------------
# 5. SNR After Enhancement
# -----------------------------
min_len = min(len(speech), len(enhanced_speech))
snr_after = compute_snr(speech[:min_len], enhanced_speech[:min_len])

print("SNR After Enhancement:", snr_after, "dB")
print("SNR Improvement:", snr_after - snr_before, "dB")

# -----------------------------
# 6. Plot Waveforms
# -----------------------------
plt.figure(figsize=(12,8))

plt.subplot(3,1,1)
plt.title("Original Speech")
plt.plot(speech)

plt.subplot(3,1,2)
plt.title("Noisy Speech (10 dB)")
plt.plot(noisy_speech)

plt.subplot(3,1,3)
plt.title("Enhanced Speech")
plt.plot(enhanced_speech)

plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'librosa'